In [1]:

import os
import sys

# Pretpostavka:
# notebook je u folderu: ergo_jr_pokretanje_ruke/notebooks/
# projekat je jedan nivo iznad
PROJECT_ROOT = os.path.abspath("..")

os.chdir(PROJECT_ROOT)
sys.path.insert(0, PROJECT_ROOT)

print("CWD:", os.getcwd())
print("PROJECT ROOT IN PATH:", PROJECT_ROOT in sys.path)

# app/main.py
import time
import cv2

from utils.debounce import Debounce
from vision.hand_open_close import HandOpenCloseDetector
from control.gripper import GripperConfig, GripperController

from IPython.display import clear_output

CAMERA_INDEX = 0  # у вас рабочий индекс


def main():
    # 1) Камера
    cap = cv2.VideoCapture(CAMERA_INDEX)
    print(cap)
    if not cap.isOpened():
        raise RuntimeError("Не удалось открыть камеру")

    # 2) Vision
    detector = HandOpenCloseDetector()

    # 3) Debounce
    deb = Debounce(stable_frames=5)

    # 4) Гриппер
    config = GripperConfig(
        motor_name="m6",   # если не уверены — проверим ниже
        open_angle=30,
        close_angle=90,
        speed=40
    )

    gripper = GripperController(config=config)
    
    gripper.connect()
    
    #gripper.poppy.rest_posture.start()
    
    p = gripper.poppy

    # Uključi motore (da drže poziciju)
    for m in p.motors:
        m.compliant = False
        m.moving_speed = 30

    # Neutralni uglovi (primer – možeš malo podesiti)
    neutral_pose = {
        "m1": 0,
        "m2": 0,
        "m3": 0,
        "m4": 0,
        "m5": 0,
        "m6": 30,  # gripper blago otvoren
    }

    for name, angle in neutral_pose.items():
        getattr(p, name).goal_position = angle

    gripper.poppy.m1.compliant = False
    gripper.poppy.m2.compliant = False
    gripper.poppy.m3.compliant = False
    gripper.poppy.m4.compliant = False
    gripper.poppy.m5.compliant = False
    gripper.poppy.m6.compliant = False

    print("Система запущена. Покажи ладонь или кулак.")

    with open("/tmp/status.log", "w") as f:
        while True:
            ret, frame = cap.read()
            if not ret:
                continue

            #gripper.close()
            #time.sleep(2)

            raw_state = detector.infer(frame)
            confirmed = deb.update(raw_state)
            clear_output(wait=True)
            print("Frame mean:", frame.mean())
            print("RAW:", raw_state)
            print("CONFIRMED:", confirmed)

            mean_val = frame.mean()

            f.write(f"{time.time():.2f} mean={mean_val:.2f} raw={raw_state} conf={confirmed}\n")
            f.flush()
            

            if confirmed == "OPEN":
                print("CONFIRMED: OPEN")
                gripper.open()

            elif confirmed == "CLOSE":
                print("CONFIRMED: CLOSE")
                gripper.close()

            time.sleep(0.02)

if __name__ == "__main__":
    main()

Frame mean: 64.50157335069444
RAW: UNKNOWN
CONFIRMED: None


KeyboardInterrupt: 